In [1]:
!pip install kaggle
import os
import json
import gc
from zipfile import ZipFile

import cv2
import keras
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.merge import concatenate
from keras.layers.pooling import MaxPooling2D

from keras.models import Model

from keras.callbacks import ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator

pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [2]:
os.environ['KAGGLE_USERNAME'] = "varunyadav17"
os.environ['KAGGLE_KEY'] = "2073c7a8d526d9d351df6f8f499e6080"
!kaggle competitions download -c severstal-steel-defect-detection

  0% 0.00/6.91M [00:00<?, ?B/s]
100% 6.91M/6.91M [00:00<00:00, 63.4MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
100% 1.16G/1.16G [00:11<00:00, 131MB/s]
100% 1.16G/1.16G [00:11<00:00, 110MB/s]
 96% 124M/129M [00:00<00:00, 134MB/s]
100% 129M/129M [00:01<00:00, 133MB/s]


In [0]:
with ZipFile('/content/train.csv.zip', 'r') as zipRef:
  zipRef.extractall('.')
  os.remove('/content/train.csv.zip')

In [0]:
if not os.path.exists('/content/test_images'):
  os.mkdir('/content/test_images')
with ZipFile('/content/test_images.zip', 'r') as zipRef:
  zipRef.extractall('/content/test_images')
  os.remove('/content/test_images.zip')

In [0]:
if not os.path.exists('/content/train_images'):
  os.mkdir('/content/train_images')
with ZipFile('/content/train_images.zip', 'r') as zipRef:
  zipRef.extractall('/content/train_images')
  os.remove('/content/train_images.zip')

In [0]:
train = pd.read_csv('train.csv')

In [7]:
train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [8]:
train.shape

(50272, 2)

In [0]:
train['ImageId'] = train['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train['ClassId'] = train['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train['hasMask'] = ~train['EncodedPixels'].isna()

In [10]:
print(train.head())

   ImageId_ClassId                                      EncodedPixels  \
0  0002cc93b.jpg_1  29102 12 29346 24 29602 24 29858 24 30114 24 3...   
1  0002cc93b.jpg_2                                                NaN   
2  0002cc93b.jpg_3                                                NaN   
3  0002cc93b.jpg_4                                                NaN   
4  00031f466.jpg_1                                                NaN   

         ImageId ClassId  hasMask  
0  0002cc93b.jpg       1     True  
1  0002cc93b.jpg       2    False  
2  0002cc93b.jpg       3    False  
3  0002cc93b.jpg       4    False  
4  00031f466.jpg       1    False  


In [11]:
mask_count_df = train.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(12568, 2)


,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [12]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns = ['ImageID'])
test_imgs.head()

,ImageID
0,004f40c73.jpg
1,006f39c41.jpg
2,00b7fb703.jpg
3,00bbcd9af.jpg
4,0108ce457.jpg


In [13]:
non_missing_training_idx = mask_count_df[mask_count_df['hasMask'] > 0]
non_missing_training_idx.head()

,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [0]:
def load_img(code, base, resize = True):
  path = f'{base}/{code}'
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.BGR2RGB)
  if resize:
    cv2.resize(img, (256, 256))
  return img

def validate_path(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [15]:
BATCH_SIZE = 64
def create_test_gen():
  return ImageDataGenerator(1./255).flow_from_dataframe(
      test_imgs,
      directory = 'test_images',
      x_col = 'ImageID',
      class_mode = None,
      target_size = (256, 256),
      batch_size = BATCH_SIZE,
      shuffle = False
  )

test_gen = create_test_gen()

Found 1801 validated image filenames.


**Data Generator**

In [0]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

In [0]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, list_IDs, df, target_df = None, mode = 'fit', base_path = 'train_images', batch_size = 32, dim = (256, 1600), n_channels = 1, 
               n_classes = 4, random_state = 2019, shuffle = True):
    print("inside init")
    self.dim = dim
    self.batch_size = batch_size 
    self.df = df
    self.mode = mode
    self.base_path = base_path
    self.target_df = target_df
    self.list_IDs = list_IDs
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.random_state = random_state

    self.on_epoch_end()

  def __len__(self):
    print("Inside len")
    return int(np.floor(len(self.list_IDs)/ self.batch_size))
  
  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index + 1)*self.batch_size]

    # Find list of IDs
    list_IDs_batch = [self.list_IDs[k] for k in indexes]

    X = self.__generate_X(list_IDs_batch)

    if self.mode == 'fit':
      y = self.__generate_y(list_IDs_batch)
      return X,y

    elif self.mode == 'predict':
      return X

    else:
      raise AttributeError('The model parameter should be set to "fit" or "predict"')

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.seed(self.random_state)
      np.random.shuffle(self.indexes)

  def __generate_X(self, list_IDs_batch):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    
    for i, ID in enumerate(list_IDs_batch):
      im_name = self.df['ImageId'].iloc[ID]
      img_path = f"{self.base_path}/{im_name}"
      img = self.__load_grayscale(img_path)
      
      X[i,] = img

    return X

  def __generate_y(self, list_IDs_batch):
    y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
    
    for i, ID in enumerate(list_IDs_batch):
      im_name = self.df['ImageId'].iloc[ID]
      image_df = self.target_df[self.target_df['ImageId'] == im_name]
      
      rles = image_df['EncodedPixels'].values
      masks = build_masks(rles, input_shape=self.dim)
      
      y[i, ] = masks

    return y
    
  def __load_grayscale(self, img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = img.astype(np.float32) / 255.
    img = np.expand_dims(img, axis=-1)

    return img

  def __load_rgb(self, img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.

    return img

In [18]:
BATCH_SIZE = 16

train_idx, val_idx = train_test_split(non_missing_training_idx.index, random_state = 2019, test_size = 0.15)
train_generator = DataGenerator(train_idx, df = mask_count_df, target_df = train, batch_size = BATCH_SIZE, n_classes = 4)
val_generator = DataGenerator(val_idx, df = mask_count_df, target_df = train, batch_size = BATCH_SIZE, n_classes = 4)

inside init
inside init


**Model**

In [0]:
def dice_coeff(y_true, y_pred, smooth = 1):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [0]:
def build_model(input_shape):
  inputs = Input(input_shape)
  
  c1 = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(inputs)
  c1 = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(c1)
  p1 = MaxPooling2D((2,2))(c1)

  c2 = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(p1)
  c2 = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(c2)
  p2 = MaxPooling2D((2,2))(c2)

  c3 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(p2)
  c3 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(c3)
  p3 = MaxPooling2D((2,2))(c3)

  c4 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(p3)
  c4 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(c4)
  p4 = MaxPooling2D((2,2))(c4)

  c5 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(p4)
  c5 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(c5)
  p5 = MaxPooling2D((2,2))(c5)

  c55 = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(p5)
  c55 = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(c55)
  
  u6 = Conv2DTranspose(64, (2,2), strides = (2,2), padding = 'same')(c55)
  u6 = concatenate([u6, c5])
  c6 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(u6)
  c6 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(c6)

  u71 = Conv2DTranspose(32, (2,2), strides = (2,2), padding = 'same')(c6)
  u71 = concatenate([u71, c4])
  c71 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(u71)
  c61 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(c71)

  u7 = Conv2DTranspose(32, (2,2), strides = (2,2), padding = 'same')(c61)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(u7)
  c7 = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(c7)

  u8 = Conv2DTranspose(16, (2,2), strides = (2,2), padding = 'same')(c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(u8)
  c8 = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(c8)

  u9 = Conv2DTranspose(8, (2,2), strides = (2,2), padding = 'same')(c8)
  u9 = concatenate([u9, c1], axis = 3)
  c9 = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(u9)
  c9 = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(c9)

  outputs = Conv2D(4, (1,1), activation = 'sigmoid')(c9)

  model = Model(inputs = [inputs], outputs = [outputs])
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [dice_coeff])

  return model

In [21]:
model = build_model((256, 1600, 1))
model.summary()







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 1600, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 8) 80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 1600, 8) 584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 800, 8)  0           conv2d_2[0][0]            

In [0]:
checkpoint = ModelCheckpoint('model.h5', monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'auto')
history = model.fit_generator(train_generator, validation_data = val_generator, callbacks = [checkpoint], use_multiprocessing = False, workers = 1, 
                              epochs = 10)

Inside len



Inside len
Inside len
Inside len
Epoch 1/10Inside len






354/354 [==============================] - 11165s 32s/step - loss: 0.0979 - dice_coeff: 0.0246 - val_loss: 0.0421 - val_dice_coeff: 0.0161
Epoch 2/10
354/354 [==============================] - 11287s 32s/step - loss: 0.0371 - dice_coeff: 0.0397 - val_loss: 0.0341 - val_dice_coeff: 0.0523
Epoch 3/10
312/354 [=========================>....] - ETA: 22:21 - loss: 0.0334 - dice_coeff: 0.0723